# <font color="red">Solar Radiation Prediction with Gradient Boosting Regressor</font>

# Library Import

In [1]:
# Data manipulation and visualization
import pandas as pd
import math
import json

# Preprocessing
from sklearn.model_selection import StratifiedShuffleSplit, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

import sys
# sys.path.append('../../') # Uncomment this line if running locally
sys.path.append('/kaggle/input/weatherdata') # Uncomment this line if running on Kaggle
from historyManagement import *

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Datasets Import

In [2]:
# Load the weather dataset for visibility prediction
# weather_data = pd.read_csv('../../FinalDatasets/finalDataset.csv') # Uncomment this line if running locally
weather_data = pd.read_csv('/kaggle/input/weatherdata/finalDataset.csv') # Uncomment this line if running on Kaggle
weather_data['DateTime'] = pd.to_datetime(weather_data['DateTime'])
weather_data.head()

,DateTime,Year,Month,Day,Hour,DaySegments,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,...,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy,Conditions,Icon
0,2023-01-01 00:00:00,2023,1,1,0,Late Night,0,0,0,1,...,0.49,1018.68,0.03,0.0,10.0,2.76,0.0,0.0,Clear,clear-night
1,2023-01-01 01:00:00,2023,1,1,1,Late Night,0,0,0,1,...,0.54,1018.03,0.11,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
2,2023-01-01 02:00:00,2023,1,1,2,Late Night,0,0,0,1,...,30.51,1017.56,0.03,0.0,10.0,1.75,0.0,0.0,Clear,clear-night
3,2023-01-01 03:00:00,2023,1,1,3,Late Night,0,0,0,1,...,49.23,1018.05,0.00,0.0,10.0,2.28,0.0,0.0,Clear,clear-night
4,2023-01-01 04:00:00,2023,1,1,4,Late Night,0,0,0,1,...,49.90,1018.00,86.17,0.0,10.0,1.27,0.0,0.0,Partially cloudy,fog


In [3]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

In [4]:
weather_data.describe()

,DateTime,Year,Month,Day,Hour,DaySegments_Afternoon,DaySegments_Early Morning,DaySegments_Evening,DaySegments_Late Night,DaySegments_Midday,...,Windgust,WindSpeed,WindDir,SeaLevelPressure,CloudCover,UVIndex,SevereRisk,Visibility,SolarRadiation,SolarEnergy
count,14616,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,...,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000,14616.000000
mean,2023-11-01 11:30:00,2023.400657,5.720854,15.735632,11.500000,0.131294,0.060550,0.098043,0.249726,0.116585,...,17.068560,8.090783,159.739178,1008.265012,53.579923,2.263555,16.504787,4.368134,227.640683,0.819217
min,2023-01-01 00:00:00,2023.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.660000,0.000000,0.000000,982.420000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,2023-06-02 05:45:00,2023.000000,3.000000,8.000000,5.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.170000,2.870000,80.965000,1003.277500,26.670000,0.000000,10.000000,3.990000,0.000000,0.000000
50%,2023-11-01 11:30:00,2023.000000,6.000000,16.000000,11.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,14.645000,7.670000,163.790000,1007.970000,51.540000,0.000000,10.000000,4.650000,10.665000,0.010000
75%,2024-04-01 17:15:00,2024.000000,8.000000,23.000000,17.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.410000,11.750000,241.402500,1013.400000,87.790000,4.740000,12.580000,4.650000,459.745000,1.660000
max,2024-08-31 23:00:00,2024.000000,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,84.120000,71.970000,360.000000,1022.060000,100.000000,10.000000,97.740000,24.100000,1026.650000,3.700000
std,NaN,0.490048,3.195073,8.803610,6.922423,0.337734,0.238511,0.297383,0.432869,0.320936,...,10.195911,6.250203,104.422027,6.010683,35.166637,3.083074,15.031827,1.376994,307.219387,1.106444


# Data Split

In [5]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for train_idx, test_idx in sss.split(weather_data, weather_data['DaySegments']):
    weather_data_train = weather_data.iloc[train_idx]
    weather_data_test = weather_data.iloc[test_idx]

In [6]:
# Show the count of each unique class in DaySegments
class_counts = weather_data['DaySegments'].value_counts().reset_index()
class_counts.columns = ['DaySegments', 'Count']

print("Distribution of data based on DaySegments:")
print(class_counts)

Distribution of data based on DaySegments:
     DaySegments  Count
0     Late Night   3650
1        Morning   3504
2      Afternoon   1919
3         Midday   1704
4          Night   1521
5        Evening   1433
6  Early Morning    885


In [7]:
print("Distribution of DaySegments in Training Set:")
print(weather_data_train['DaySegments'].value_counts())

print("\nDistribution of DaySegments in Test Set:")
print(weather_data_test['DaySegments'].value_counts())

Distribution of DaySegments in Training Set:
DaySegments
Late Night       2920
Morning          2803
Afternoon        1535
Midday           1363
Night            1217
Evening          1146
Early Morning     708
Name: count, dtype: int64

Distribution of DaySegments in Test Set:
DaySegments
Late Night       730
Morning          701
Afternoon        384
Midday           341
Night            304
Evening          287
Early Morning    177
Name: count, dtype: int64


# Some Reused Parameters

In [8]:
grid_param = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6],
    'subsample': [0.7, 0.8, 0.9],
    'max_features': ['sqrt', 'log2', None],
    'loss': ['squared_error', 'huber']
}

In [9]:
number_of_model_for_randomized_grid = 100 # Estimated run time 10 hours
model_name_for_saving = "Gradient Boosting"
target_name_for_saving = "SolarRadiation"

# All Features

In [10]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_train = weather_data_train['SolarRadiation']

In [11]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon']) # Target variable
y_test = weather_data_test['SolarRadiation']

In [12]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [13]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


## MinMax Scaler

In [14]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [15]:
X_train

array([[0.72727273, 0.83333333, 0.43478261, ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.26666667, 0.69565217, ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.43333333, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.86666667, 0.43478261, ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.4       , 0.91304348, ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.26666667, 0.34782609, ..., 0.8925    , 0.29      ,
        0.27812962]])

In [16]:
X_test

array([[0.63636364, 0.06666667, 0.95652174, ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.26666667, 0.65217391, ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.03333333, 0.2173913 , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.2       , 0.73913043, ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.13333333, 0.86956522, ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.73333333, 0.95652174, ..., 0.5161    , 0.        ,
        0.07388643]])

### Default Parameters

In [17]:
training_model = GradientBoostingRegressor(random_state=42)
training_model

GradientBoostingRegressor(random_state=42)

In [18]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 4.21 s, sys: 0 ns, total: 4.21 s
Wall time: 4.23 s


GradientBoostingRegressor(random_state=42)

In [19]:
y_pred = training_model.predict(X_test)
y_pred

array([  0.4414199 ,  69.40394919,  -0.74075654, ..., 199.62051153,
         0.57049164,   0.74520588])

In [20]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [21]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [22]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 251.64913459986803
MAE Score: 9.97040821285509
R2 Score: 0.9973239782688688
RMSE Score: 15.863452795651646


In [23]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 251.649135, R²: 0.997324
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Randomized Grid Search

In [24]:
grid_search = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [25]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: user 1min 5s, sys: 12.2 s, total: 1min 17s
Wall time: 1h 44min 59s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [26]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 10, 'loss': 'huber', 'learning_rate': 0.05}
Best Cross-Validation Score (Negative MSE): -193.9605384049217
Best Cross-Validation Score (MSE): 193.9605384049217
Best Cross-Validation Score (RMSE): 13.926971616432686


In [27]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [28]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [29]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 203.09165868127252
MAE Score: 8.054450638092268
R2 Score: 0.9978403355413611
RMSE Score: 14.251023074897905


In [30]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 203.091659, R²: 0.997840
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Standard Scaler

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [32]:
X_train

array([[ 1.01884116,  1.17229173, -0.21938027, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461, -0.75947244,  0.64624572, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -0.1913065 , -1.66209024, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943,  1.28592492, -0.21938027, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264, -0.30493969,  1.36760071, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.75947244, -0.50792226, ...,  1.01722469,
         0.20430944,  0.85928021]])

In [33]:
X_test

array([[ 0.70637943, -1.44127155,  1.51187171, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598, -0.75947244,  0.50197472, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -1.55490474, -0.94073525, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092, -0.98673881,  0.79051672, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747, -1.21400518,  1.22332971, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  0.83139217,  1.51187171, ..., -0.05293696,
        -0.73592134, -0.43343649]])

### Default Parameters

In [34]:
training_model = GradientBoostingRegressor(random_state=42)
training_model

GradientBoostingRegressor(random_state=42)

In [35]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 4.21 s, sys: 0 ns, total: 4.21 s
Wall time: 4.23 s


GradientBoostingRegressor(random_state=42)

In [36]:
y_pred = training_model.predict(X_test)
y_pred

array([  0.4414199 ,  69.40394919,  -0.74075654, ..., 199.62051153,
         0.57049164,   0.74520588])

In [37]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [38]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [39]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 251.64898608880236
MAE Score: 9.969150127587435
R2 Score: 0.9973239798481265
RMSE Score: 15.863448114732256


In [40]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 251.648986, R²: 0.997324
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [41]:
grid_search = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [42]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: user 2min 28s, sys: 9.51 s, total: 2min 38s
Wall time: 1h 45min 35s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [43]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'subsample': 0.7, 'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 30, 'loss': 'huber', 'learning_rate': 0.05}
Best Cross-Validation Score (Negative MSE): -195.2595663244082
Best Cross-Validation Score (MSE): 195.2595663244082
Best Cross-Validation Score (RMSE): 13.97353091829006


In [44]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [45]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [46]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 207.37154833782805
MAE Score: 8.057887108372771
R2 Score: 0.9977948234526904
RMSE Score: 14.400400978369596


In [47]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 207.371548, R²: 0.997795
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


## Robust Scaler

In [48]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [49]:
X_train

array([[ 0.6       ,  0.66666667, -0.16666667, ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        , -0.46666667,  0.33333333, ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -0.13333333, -1.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       ,  0.73333333, -0.16666667, ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        , -0.2       ,  0.75      , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.46666667, -0.33333333, ...,  0.61755563,
         0.59917355,  7.5       ]])

In [50]:
X_test

array([[ 4.00000000e-01, -8.66666667e-01,  8.33333333e-01, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -4.66666667e-01,  2.50000000e-01, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -9.33333333e-01, -5.83333333e-01, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01, -6.00000000e-01,  4.16666667e-01, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01, -7.33333333e-01,  6.66666667e-01, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  4.66666667e-01,  8.33333333e-01, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]])

### Default Parameters

In [51]:
training_model = GradientBoostingRegressor(random_state=42)
training_model

GradientBoostingRegressor(random_state=42)

In [52]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 4.22 s, sys: 0 ns, total: 4.22 s
Wall time: 4.23 s


GradientBoostingRegressor(random_state=42)

In [53]:
y_pred = training_model.predict(X_test)
y_pred

array([  0.4414199 ,  69.40394919,  -0.74075654, ..., 199.62051153,
         0.57049164,   0.74520588])

In [54]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [55]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [56]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 251.63780031154667
MAE Score: 9.968920471960097
R2 Score: 0.9973240987970077
RMSE Score: 15.86309554631588


In [57]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 251.637800, R²: 0.997324
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


### Tuning with Grid Search

In [58]:
grid_search = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [59]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: user 1min 7s, sys: 8 s, total: 1min 15s
Wall time: 1h 43min 52s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [60]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'min_samples_split': 20, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 10, 'loss': 'huber', 'learning_rate': 0.05}
Best Cross-Validation Score (Negative MSE): -194.9571799392692
Best Cross-Validation Score (MSE): 194.9571799392692
Best Cross-Validation Score (RMSE): 13.962706755470775


In [61]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [62]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [63]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 203.44213202221198
MAE Score: 8.03201852268365
R2 Score: 0.9978366086289756
RMSE Score: 14.263314201903146


In [64]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

✨ New regression model added:
   MSE: 203.442132, R²: 0.997837
   Features: 27
💾 Registry updated: Model_Training_History/History_Regression.csv


# Selected Features

In [65]:
numeric_cols = weather_data.select_dtypes(include=[float, int]).columns
correlations = weather_data[numeric_cols].corr()['SolarRadiation'].sort_values(ascending=False)
print(correlations)

SolarRadiation               1.000000
SolarEnergy                  0.999867
UVIndex                      0.998315
DaySegments_Midday           0.594891
Temp                         0.400378
FeelsLike                    0.344429
DaySegments_Morning          0.244659
WindSpeed                    0.216803
DaySegments_Afternoon        0.215785
WindDir                      0.180924
SevereRisk                   0.125829
Hour                         0.095598
Season_Summer                0.093764
Visibility                   0.054367
SeaLevelPressure             0.026622
Year                         0.021999
Dew                          0.013910
Day                         -0.001404
PrecipProb                  -0.013477
Season_Rainy                -0.033059
Season_Winter               -0.035395
Season_Autumn               -0.035413
Precip                      -0.039079
CloudCover                  -0.055418
Month                       -0.061339
Windgust                    -0.100270
DaySegments_

In [66]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14616 entries, 0 to 14615
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DateTime                   14616 non-null  datetime64[ns]
 1   Year                       14616 non-null  int64         
 2   Month                      14616 non-null  int64         
 3   Day                        14616 non-null  int64         
 4   Hour                       14616 non-null  int64         
 5   DaySegments                14616 non-null  object        
 6   DaySegments_Afternoon      14616 non-null  int64         
 7   DaySegments_Early Morning  14616 non-null  int64         
 8   DaySegments_Evening        14616 non-null  int64         
 9   DaySegments_Late Night     14616 non-null  int64         
 10  DaySegments_Midday         14616 non-null  int64         
 11  DaySegments_Morning        14616 non-null  int64         
 12  DayS

## Run Again

In [67]:
X_train_raw = weather_data_train.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Energy
                                                'Dew', # low correlation with Solar Energy
                                                'Precip', # low correlation with Solar Energy
                                                'PrecipProb']) # low correlation with Solar Energy
y_train = weather_data_train['SolarRadiation']

In [68]:
X_test_raw = weather_data_test.drop(columns=['DateTime', # Model cannot use DateTime as a feature
                                                'Year', # No Effect on weather data
                                                'Season', # Season is categorical, not numerical
                                                'DaySegments', # DaySegments is categorical, not numerical
                                                'Visibility',  # Target variable
                                                'SolarRadiation', # Target variable
                                                'SolarEnergy', # Target variable
                                                'Conditions', # Target variable
                                                'Icon', # Target variable
                                                
                                                'Day', # same day from different month act differently
                                                'SeaLevelPressure', # low correlation with Solar Energy
                                                'Dew', # low correlation with Solar Energy
                                                'Precip', # low correlation with Solar Energy
                                                'PrecipProb']) # low correlation with Solar Energy
y_test = weather_data_test['SolarRadiation']

In [69]:
feature_columns = [col for col in X_train_raw.columns]
feature_columns_for_saving = ','.join(feature_columns)

In [70]:
print("X_train: ", len(X_train_raw))
print("y_train: ", len(y_train))
print("\nX_test: ", len(X_test_raw))
print("y_test: ", len(y_test))

X_train:  11692
y_train:  11692

X_test:  2924
y_test:  2924


### MinMax Scaler

In [71]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [72]:
X_train

array([[0.72727273, 0.43478261, 0.        , ..., 0.4922    , 0.697     ,
        0.31560059],
       [0.90909091, 0.69565217, 1.        , ..., 0.        , 0.303     ,
        0.07388643],
       [0.09090909, 0.        , 0.        , ..., 0.        , 0.        ,
        0.07388643],
       ...,
       [0.63636364, 0.43478261, 0.        , ..., 0.8756    , 0.306     ,
        0.09436352],
       [0.        , 0.91304348, 0.        , ..., 0.        , 0.        ,
        0.07388643],
       [0.72727273, 0.34782609, 0.        , ..., 0.8925    , 0.29      ,
        0.27812962]])

In [73]:
X_test

array([[0.63636364, 0.95652174, 0.        , ..., 0.894     , 0.        ,
        0.07388643],
       [0.45454545, 0.65217391, 1.        , ..., 0.9677    , 0.074     ,
        0.07388643],
       [0.63636364, 0.2173913 , 0.        , ..., 0.9935    , 0.        ,
        0.07388643],
       ...,
       [0.09090909, 0.73913043, 1.        , ..., 0.0013    , 0.203     ,
        0.07388643],
       [0.27272727, 0.86956522, 0.        , ..., 0.        , 0.        ,
        0.25575259],
       [0.45454545, 0.95652174, 0.        , ..., 0.5161    , 0.        ,
        0.07388643]])

#### Default Parameters

In [74]:
training_model = GradientBoostingRegressor(random_state=42)
training_model

GradientBoostingRegressor(random_state=42)

In [75]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 3.43 s, sys: 0 ns, total: 3.43 s
Wall time: 3.48 s


GradientBoostingRegressor(random_state=42)

In [76]:
y_pred = training_model.predict(X_test)
y_pred

array([  0.37673166,  69.19516244,  -1.19870015, ..., 199.63957923,
         0.46911452,   0.45702301])

In [77]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [78]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [79]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 252.86657646997307
MAE Score: 9.998139201449364
R2 Score: 0.997311032065394
RMSE Score: 15.901779034748692


In [80]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 251.649135 ≈ 252.866576 (similar)
   R²:  0.997324 ≈ 0.997311 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [81]:
grid_search = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [82]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: user 45.7 s, sys: 9.72 s, total: 55.4 s
Wall time: 1h 35min 39s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [83]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 6, 'max_features': None, 'max_depth': 10, 'loss': 'huber', 'learning_rate': 0.05}
Best Cross-Validation Score (Negative MSE): -200.9829913000155
Best Cross-Validation Score (MSE): 200.9829913000155
Best Cross-Validation Score (RMSE): 14.176847015469113


In [84]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [85]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [86]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 210.50531422979364
MAE Score: 8.337504918111483
R2 Score: 0.9977614991750589
RMSE Score: 14.508801267844069


In [87]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "MinMaxScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 210.505314, Best MSE: 203.091659
   Current R²: 0.997761, Best R²: 0.997840
   Current Features: 22, Best Features: 27


### Standard Scaler

In [88]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [89]:
X_train

array([[ 1.01884116, -0.21938027, -0.38875095, ..., -0.12088825,
         1.52387472,  1.09644529],
       [ 1.64376461,  0.64624572,  2.57234098, ..., -1.52028666,
         0.24645772, -0.43343649],
       [-1.16839092, -1.66209024, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       ...,
       [ 0.70637943, -0.21938027, -0.38875095, ...,  0.96917545,
         0.25618424, -0.30383078],
       [-1.48085264,  1.36760071, -0.38875095, ..., -1.52028666,
        -0.73592134, -0.43343649],
       [ 1.01884116, -0.50792226, -0.38875095, ...,  1.01722469,
         0.20430944,  0.85928021]])

In [90]:
X_test

array([[ 0.70637943,  1.51187171, -0.38875095, ...,  1.02148941,
        -0.73592134, -0.43343649],
       [ 0.08145598,  0.50197472,  2.57234098, ...,  1.23102956,
        -0.49600038, -0.43343649],
       [ 0.70637943, -0.94073525, -0.38875095, ...,  1.30438283,
        -0.73592134, -0.43343649],
       ...,
       [-1.16839092,  0.79051672,  2.57234098, ..., -1.51659057,
        -0.07775979, -0.43343649],
       [-0.54346747,  1.22332971, -0.38875095, ..., -1.52028666,
        -0.73592134,  0.71764924],
       [ 0.08145598,  1.51187171, -0.38875095, ..., -0.05293696,
        -0.73592134, -0.43343649]])

#### Default Parameters

In [91]:
training_model = GradientBoostingRegressor(random_state=42)
training_model

GradientBoostingRegressor(random_state=42)

In [92]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 3.27 s, sys: 0 ns, total: 3.27 s
Wall time: 3.27 s


GradientBoostingRegressor(random_state=42)

In [93]:
y_pred = training_model.predict(X_test)
y_pred

array([  0.37673166,  69.19516244,  -1.19870015, ..., 199.63957923,
         0.46911452,   0.45702301])

In [94]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [95]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [96]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 252.8277766573784
MAE Score: 9.99589160139764
R2 Score: 0.9973114446602627
RMSE Score: 15.900559004556362


In [97]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 251.648986 ≈ 252.827777 (similar)
   R²:  0.997324 ≈ 0.997311 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [98]:
grid_search = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [99]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: user 45.5 s, sys: 10.2 s, total: 55.7 s
Wall time: 1h 35min 52s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [100]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 6, 'max_features': None, 'max_depth': 10, 'loss': 'huber', 'learning_rate': 0.05}
Best Cross-Validation Score (Negative MSE): -200.12388483468675
Best Cross-Validation Score (MSE): 200.12388483468675
Best Cross-Validation Score (RMSE): 14.146514936007623


In [101]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [102]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [103]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 210.5365327152124
MAE Score: 8.333550930430448
R2 Score: 0.9977611671995663
RMSE Score: 14.50987707443493


In [104]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "StandardScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 210.536533, Best MSE: 207.371548
   Current R²: 0.997761, Best R²: 0.997795
   Current Features: 22, Best Features: 27


### Robust Scaler

In [105]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

In [106]:
X_train

array([[ 0.6       , -0.16666667,  0.        , ..., -0.03738547,
         1.44008264,  8.87596899],
       [ 1.        ,  0.33333333,  1.        , ..., -0.84268652,
         0.62603306,  0.        ],
       [-0.8       , -1.        ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       ...,
       [ 0.4       , -0.16666667,  0.        , ...,  0.5899051 ,
         0.6322314 ,  0.75193798],
       [-1.        ,  0.75      ,  0.        , ..., -0.84268652,
         0.        ,  0.        ],
       [ 0.6       , -0.33333333,  0.        , ...,  0.61755563,
         0.59917355,  7.5       ]])

In [107]:
X_test

array([[ 4.00000000e-01,  8.33333333e-01,  0.00000000e+00, ...,
         6.20009817e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.50000000e-01,  1.00000000e+00, ...,
         7.40592277e-01,  1.52892562e-01,  0.00000000e+00],
       [ 4.00000000e-01, -5.83333333e-01,  0.00000000e+00, ...,
         7.82804319e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.00000000e-01,  4.16666667e-01,  1.00000000e+00, ...,
        -8.40559555e-01,  4.19421488e-01,  0.00000000e+00],
       [-4.00000000e-01,  6.66666667e-01,  0.00000000e+00, ...,
        -8.42686518e-01,  0.00000000e+00,  6.67829457e+00],
       [ 0.00000000e+00,  8.33333333e-01,  0.00000000e+00, ...,
         1.71793194e-03,  0.00000000e+00,  0.00000000e+00]])

#### Default Parameters

In [108]:
training_model = GradientBoostingRegressor(random_state=42)
training_model

GradientBoostingRegressor(random_state=42)

In [109]:
%%time
training_model.fit(X_train, y_train)

CPU times: user 3.29 s, sys: 0 ns, total: 3.29 s
Wall time: 3.29 s


GradientBoostingRegressor(random_state=42)

In [110]:
y_pred = training_model.predict(X_test)
y_pred

array([  0.37673166,  69.19516244,  -1.19870015, ..., 199.63957923,
         0.46911452,   0.45702301])

In [111]:
y_test

5158       0.00
3831     105.26
5117       0.00
6326     758.39
11359    135.87
          ...  
10290     74.58
13995      0.00
9665     241.81
2276       0.00
4174       0.00
Name: SolarRadiation, Length: 2924, dtype: float64

In [112]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mse)

In [113]:
print("MSE Score:", mse)
print("MAE Score:", mae)
print("R2 Score:", r2)
print("RMSE Score:", rmse)

MSE Score: 252.8277766573784
MAE Score: 9.99589160139764
R2 Score: 0.9973114446602627
RMSE Score: 15.900559004556362


In [114]:
regression_params = {
    'model_name': model_name_for_saving,
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'parameters': json.dumps(training_model.get_params())
}
save_model_performance_if_better('regression', regression_params)

✅ Regression model improved with same performance but fewer features!
   MSE: 251.637800 ≈ 252.827777 (similar)
   R²:  0.997324 ≈ 0.997311 (similar)
   Features: 27 → 22 (🎯 5 fewer features!)
💾 Registry updated: Model_Training_History/History_Regression.csv


#### Tuning with Grid Search

In [115]:
grid_search = RandomizedSearchCV(
    GradientBoostingRegressor(random_state=42), 
    grid_param,
    n_iter=number_of_model_for_randomized_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [116]:
%%time
print("Starting hyperparameter tuning...")
grid_search.fit(X_train, y_train)

Starting hyperparameter tuning...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: user 45.2 s, sys: 9.75 s, total: 54.9 s
Wall time: 1h 34min 58s


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [117]:
print("TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score (Negative MSE):", grid_search.best_score_)
print("Best Cross-Validation Score (MSE):", -grid_search.best_score_)
print("Best Cross-Validation Score (RMSE):", math.sqrt(-grid_search.best_score_))

TUNING RESULTS
Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 6, 'max_features': None, 'max_depth': 10, 'loss': 'huber', 'learning_rate': 0.05}
Best Cross-Validation Score (Negative MSE): -200.5919485966612
Best Cross-Validation Score (MSE): 200.5919485966612
Best Cross-Validation Score (RMSE): 14.163048704168931


In [118]:
grid_search_best_model = grid_search.best_estimator_
y_pred = grid_search_best_model.predict(X_test)

In [119]:
mse_grid_search = mean_squared_error(y_test, y_pred)
mae_grid_search = mean_absolute_error(y_test, y_pred)
r2_grid_search = r2_score(y_test, y_pred)
rmse_grid_search = math.sqrt(mse_grid_search)

In [120]:
print("MSE Score:", mse_grid_search)
print("MAE Score:", mae_grid_search)
print("R2 Score:", r2_grid_search)
print("RMSE Score:", rmse_grid_search)

MSE Score: 211.69211249421537
MAE Score: 8.334129500910041
R2 Score: 0.99774887883384
RMSE Score: 14.549643036659537


In [121]:
regression_params = {
    'model_name': model_name_for_saving + " Tuned",
    'scaler_name': "RobustScaler",
    'features_list': feature_columns_for_saving,
    'target_column': target_name_for_saving,
    'mse': mse_grid_search,
    'mae': mae_grid_search,
    'rmse': rmse_grid_search,
    'r2': r2_grid_search,
    'parameters' : json.dumps({
        "best_params": grid_search.best_params_,
        "cv_score": grid_search.best_score_,
        "all_params": grid_search_best_model.get_params()
    })
}
save_model_performance_if_better('regression', regression_params)

❌ Regression model not improved:
   Current MSE: 211.692112, Best MSE: 203.442132
   Current R²: 0.997749, Best R²: 0.997837
   Current Features: 22, Best Features: 27


# All Performance

In [122]:
show_model_history('regression', model_name=model_name_for_saving, target_column=target_name_for_saving)


📊 REGRESSION Model Performance History
                  Model         Scaler         Target  Features        MSE      MAE      RMSE       R²
Gradient Boosting Tuned   MinMaxScaler SolarRadiation        27 203.091659 8.054451 14.251023 0.997840
Gradient Boosting Tuned   RobustScaler SolarRadiation        27 203.442132 8.032019 14.263314 0.997837
Gradient Boosting Tuned StandardScaler SolarRadiation        27 207.371548 8.057887 14.400401 0.997795
      Gradient Boosting StandardScaler SolarRadiation        22 252.827777 9.995892 15.900559 0.997311
      Gradient Boosting   RobustScaler SolarRadiation        22 252.827777 9.995892 15.900559 0.997311
      Gradient Boosting   MinMaxScaler SolarRadiation        22 252.866576 9.998139 15.901779 0.997311

📈 Total models shown: 6
🏆 Best R² Score: 0.997840 (Gradient Boosting Tuned + MinMaxScaler for SolarRadiation)


# <center><font size="50" color="red">Thank You</font></center>